In [ ]:
"""
NER_CRF
# Adapted from Medium (https://towardsdatascience.com/a-review-of-named-entity-recognition-ner-using-automatic-summarization-of-resumes-5248a75de175).
# Implemented  by Lavanya Thollamadugu
"""

from google.colab import drive
drive.mount('/content/drive')

resume_JSON_FilePath = '/content/drive/My Drive/Entity Recognition in Resumes.json''

import numpy as np 
import pandas as pd
import json
import re

import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
!pip install seqeval==0.0.12

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for seqeval: filename=seqeval-0.0.12-py3-none-any.whl size=7434 sha256=010939621423362acacd88cb83865ba75a917d586b2b2352bf13a611f34ba8a1
  Stored in directory: /root/.cache/pip/wheels/dc/cc/62/a3b81f92d35a80e39eb9b2a9d8b31abac54c02b21b2d466edc
Successfully built seqeval


In [ ]:
!pip install spacy==2.1.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.8 MB 1.4 MB/s 
     |████████████████████████████████| 82 kB 534 kB/s 
     |████████████████████████████████| 2.1 MB 51.4 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 208 kB 52.0 MB/s 
     |████████████████████████████████| 3.2 MB 42.6 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.5
    Uninstalling srsly-2.4.5:
      Successfully uninstalled srsly-2.4.5
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.8
    Uninstalling preshed-3.0.8:
      Successfully uninstalled preshed-3.0.8
  Attempting uninstall: blis
    Found existing installation: blis 0.7.9
    Uninstalling blis-0.7.9:
      Successfully uninstalled blis-0.7.9
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.5
    Uninstalling thinc-8.1.5:
    

In [ ]:
def span_trim_entity(data: list) -> list:
    invalid_tokens = re.compile(r'\s')

    clean_data = []
    for resume_text, resume_annotations in data:
        entities = resume_annotations['entities']
        entities_v = []
        for i, j, label in entities:
            start = i
            end = j
            while start < len(resume_text) and invalid_tokens.match(
                    resume_text[start]):
                start += 1
            while end > 1 and invalid_tokens.match(
                    resume_text[end - 1]):
                end -= 1
            entities_v.append([start, end, label])
        clean_data.append([resume_text, {'entities': entities}])
    return clean_data

**Cleaning Entities**

In [ ]:
# data formating
def map_dataset_to_spacy(resume_JSON_FilePath):
    train_data = []
    each_resume=[]
    with open(resume_JSON_FilePath, 'r') as f:
        each_resume = f.readlines()

    for resume in each_resume:
        resume_data = json.loads(resume)
        text = resume_data['content'].replace("\n", " ")
        original_entities = []
        resume_data_annotations = resume_data['annotation']
        if resume_data_annotations is not None:
            for annotation in resume_data_annotations:
                each_data_record = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    start_point = each_data_record['start']
                    end_point = each_data_record['end']
                    point_text = each_data_record['text']

                    l_diff = len(point_text) - len(point_text.lstrip())
                    r_diff = len(point_text) - len(point_text.rstrip())
                    if l_diff != 0:
                        start_point = start_point + l_diff
                    if r_diff != 0:
                        end_point = end_point - r_diff
                    original_entities.append((start_point, end_point + 1 , label))
        train_data.append((text, {"entities" : original_entities}))
    return train_data



In [ ]:
data = span_trim_entity(map_dataset_to_spacy(resume_JSON_FilePath))
data[0]

["Abhishek Jha Application Development Associate - Accenture  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a  • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company's growth in best possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK EXPERIENCE  Application Development Associate  Accenture -  November 2017 to Present  Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input. Also, Training the bot for different possible utterances (Both positive and negative), which will be given as input by the user.  EDUCATION  B.E in Information science and engineering  B.v.b college of engineering and technology -  Hubli, Karnataka  August 2013 to June 2017  12th in Mathematics  Woodbine modern school  April 2011 to March 2013  10th  Kendriya Vidyalaya  April 2001 to March 2011  SKILLS  C (Le

**Entity Mapping**

In [ ]:
from spacy.lang.en import English
from spacy.gold import biluo_tags_from_offsets

def bilou_tags_offset(data):
    
    doc  = []
    annots = []
    nlp = English()
    for text, annotations in data:
        offsets = annotations["entities"]
        docs = nlp(text)
        bilou_tags = biluo_tags_from_offsets(docs, offsets)
        for i in range(len(bilou_tags)):
            if bilou_tags[i].startswith("U"):
                bilou_tags[i] = "B" + bilou_tags[i][1:]
            elif bilou_tags[i].startswith("L"):
                bilou_tags[i] = "I" + bilou_tags[i][1:]
            if not (docs[i].text.isalnum() or len(docs[i].text) > 1):
                bilou_tags[i] = "O"
        doc.append([token.text for token in docs])
        annots.append(bilou_tags)
        
    df_data = pd.DataFrame({'docs': doc, 'annots': annots})

    return df_data

df_data = bilou_tags_offset(data)

**Remove mislabeled examples**

In [ ]:
for i in range(len(df_data)):
    if "-" in df_data.loc[i, "annots"]:
        df_data.drop(i, axis = "index", inplace = True)
df_data.reset_index(inplace = True)

In [ ]:
sentences = [[(w, p, t) for w, p, t in zip(df_data["docs"][i], [y for x, y in pos_tag(df_data["docs"][i])], df_data["annots"][i]) if w.isalnum() or len(w) > 1] for i in range(0, len(df_data))]

**Feature extraction**

In [ ]:
def word_features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2]
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2]
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2]
        })
    else:
        features['EOS'] = True
    return features
def sent_features(sent):
    return [word_features(sent, i) for i in range(len(sent))]
def sent_labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

**Split dataset**

In [ ]:
%%time
from sklearn.model_selection import train_test_split

X = [sent_features(s) for s in sentences]
y = [sent_labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

CPU times: user 153 ms, sys: 15.1 ms, total: 168 ms
Wall time: 170 ms


In [ ]:
!pip install python-crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 30.7 MB/s 


**Load model**

In [ ]:
import pycrfsuite

trainer_crf = pycrfsuite.Trainer(verbose = True)

for xsequence, ysequence in zip(X_train, y_train):
    trainer_crf.append(xsequence, ysequence)

In [ ]:
trainer_crf.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 100,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [ ]:
trainer_crf.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [ ]:
trainer_crf.train('resume-ner.crfsuite')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 30978
Seconds required: 0.185

L-BFGS optimization
c1: 1.000000
c2: 0.001000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 65089.100571
Feature norm: 1.000000
Error norm: 51585.760891
Active features: 16084
Line search trials: 1
Line search step: 0.000015
Seconds required for this iteration: 0.321

***** Iteration #2 *****
Loss: 60729.194849
Feature norm: 4.867578
Error norm: 18164.336992
Active features: 15544
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.177

***** Iteration #3 *****
Loss: 47543.208327
Feature norm: 4.281969
Error norm: 18115.521582
Active features: 13318
Line search trials: 1
Line search step: 1.000000
Seconds required for

In [ ]:
trainer_crf.logparser.last_iteration

{'num': 100,
 'scores': {},
 'loss': 3242.371492,
 'feature_norm': 45.200016,
 'error_norm': 537.135129,
 'active_features': 2920,
 'linesearch_trials': 1,
 'linesearch_step': 1.0,
 'time': 0.109}

In [ ]:
tagger_crf = pycrfsuite.Tagger()
tagger_crf.open('./resume-ner.crfsuite')

**Evaluation**

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from itertools import chain

def ner_report(y_true, y_pred):
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_)
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset
    ), accuracy_score(y_true_combined, y_pred_combined)

In [ ]:
y_pred = [tagger_crf.tag(xseq) for xseq in X_test]
report, accuracy = ner_report(y_test, y_pred)
print(report)

                       precision    recall  f1-score   support

       B-College Name       0.60      0.43      0.50        14
       I-College Name       0.64      0.67      0.65        21
B-Companies worked at       0.78      0.49      0.60        37
I-Companies worked at       0.25      0.11      0.15         9
             B-Degree       0.57      0.44      0.50         9
             I-Degree       0.40      0.50      0.44        16
        B-Designation       1.00      0.57      0.73        28
        I-Designation       0.96      0.62      0.75        39
      B-Email Address       0.78      0.78      0.78         9
      I-Email Address       0.92      0.80      0.86        15
    B-Graduation Year       0.33      0.07      0.11        15
           B-Location       0.64      0.33      0.44        21
           I-Location       0.00      0.00      0.00         4
               B-Name       1.00      1.00      1.00        11
               I-Name       1.00      0.92      0.96  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(accuracy)

0.9060324825986079


In [ ]:
from seqeval.metrics import classification_report
print(classification_report(y_test, y_pred,digits=4))

                     precision    recall  f1-score   support

Years of Experience     0.0000    0.0000    0.0000         2
Companies worked at     0.7826    0.4865    0.6000        37
    Graduation Year     0.3333    0.0667    0.1111        15
             Degree     0.4286    0.3333    0.3750         9
           Location     0.6364    0.3333    0.4375        21
      Email Address     0.7778    0.7778    0.7778         9
               Name     0.9091    0.9091    0.9091        11
       College Name     0.5000    0.3571    0.4167        14
        Designation     0.9375    0.5357    0.6818        28
             Skills     0.5714    0.3077    0.4000        13

          micro avg     0.7216    0.4403    0.5469       159
          macro avg     0.6846    0.4403    0.5255       159

